In [14]:
import os
import sys
import pandas
from typing import Union


base_path = os.getcwd()
proj_path = os.path.join(base_path, "Project_2") #/ECE219/Project_2
print(base_path)
print(proj_path)

sys.path.append(proj_path)
from pokedex_helper import *

/Users/richardlee/Desktop/UCLA/Classes/2025_Winter/ECE219/ECE219
/Users/richardlee/Desktop/UCLA/Classes/2025_Winter/ECE219/ECE219/Project_2


In [9]:
pokemon = pd.read_csv(os.path.join(proj_path, "Pokemon.csv"))
display(pokemon)

,ID,Name,Form,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,1,Bulbasaur,,Grass,Poison,318,45,49,49,65,65,45,1
1,2,Ivysaur,,Grass,Poison,405,60,62,63,80,80,60,1
2,3,Venusaur,,Grass,Poison,525,80,82,83,100,100,80,1
3,4,Charmander,,Fire,,309,39,52,43,60,50,65,1
4,5,Charmeleon,,Fire,,405,58,64,58,80,65,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,,Steel,Psychic,590,90,72,100,122,108,98,9
1211,1024,Terapagos,Normal Form,Normal,,450,90,65,85,65,85,60,9
1212,1024,Terapagos,Terastal Form,Normal,,600,95,95,110,105,110,85,9
1213,1024,Terapagos,Stellar Form,Normal,,700,160,105,110,130,110,85,9


In [ ]:
def create_text_query(category: str, labels: Union[str,list]) -> str:
    if isinstance(labels, str):
        final_labels = labels
    elif isinstance(labels, list) and labels:
        for idx, label in enumerate(labels):
            if idx == 0:
                final_labels = label
            elif idx > 0:
                final_labels += f' and {label}'
    else:
        final_labels = ""  # Handle empty list case
    return f"{category}: {final_labels}"

def parse_text_query(text_query: str):
    """Parses a text query of the format 'Category: label1 and label2'."""
    parts = text_query.split(':', 1)  # Split only by the first colon
    if len(parts) < 2:
        raise ValueError("Invalid query format. Expected format: 'Category: label1 and label2'")

    category = parts[0].strip()  # Extract and clean category
    labels = [label.strip() for label in parts[1].split('and')]  # Clean each label
    return category, labels

def process_query(text_query: str, data: pd.DataFrame):
    """Retrieve image file paths from text query"""
    category, labels = parse_text_query(text_query)
    pokemond_names = []
    if category.lower() == "type" and len(labels) == 2:
        pokemond_names = 
        





print(create_text_query("Type", ["fire", "dragon"]))
print(parse_text_query(create_text_query("Type", ["fire", "dragon"])))

SyntaxError: expected ':' (2354464815.py, line 24)